# 使用 LLaMA Factory 微调 Llama-3 中文对话模型

请申请一个免费 T4 GPU 来运行该脚本

项目主页: https://github.com/hiyouga/LLaMA-Factory


## 安装 LLaMA Factory 依赖

In [ ]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install -e .[torch,bitsandbytes]

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 16155, done.
remote: Counting objects: 100% (6973/6973), done.
remote: Compressing objects: 100% (383/383), done.
remote: Total 16155 (delta 6637), reused 6647 (delta 6587), pack-reused 9182
Receiving objects: 100% (16155/16155), 222.75 MiB | 19.36 MiB/s, done.
Resolving deltas: 100% (12009/12009), done.
/content/LLaMA-Factory
assets/       docker/      LICENSE      pyproject.toml  requirements.txt  src/
CITATION.cff  evaluation/  Makefile     README.md       scripts/          tests/
data/         examples/    MANIFEST.in  README_zh.md    setup.py
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━

### 检查 GPU 环境

免费 T4 申请教程：https://zhuanlan.zhihu.com/p/642542618

In [ ]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("需要 GPU 环境，申请教程：https://zhuanlan.zhihu.com/p/642542618")

## 更新自我认知数据集

可以自由修改 NAME 和 AUTHOR 变量的内容。

In [ ]:
import json

%cd /content/LLaMA-Factory/

NAME = "Llama-Chinese"
AUTHOR = "LLaMA Factory"

with open("data/identity.json", "r", encoding="utf-8") as f:
  dataset = json.load(f)

for sample in dataset:
  sample["output"] = sample["output"].replace("{{"+ "name" + "}}", NAME).replace("{{"+ "author" + "}}", AUTHOR)

with open("data/identity.json", "w", encoding="utf-8") as f:
  json.dump(dataset, f, indent=2, ensure_ascii=False)

/content/LLaMA-Factory


## 使用 LLaMA Board Web UI 微调模型

In [ ]:
%cd /content/LLaMA-Factory/data/
!wget https://huggingface.co/datasets/ChenWeiLi/Medtext_zhtw/raw/main/MedText_zhtw.json


/content/LLaMA-Factory/data
--2024-08-13 01:43:18--  https://huggingface.co/datasets/ChenWeiLi/Medtext_zhtw/raw/main/MedText_zhtw.json
Resolving huggingface.co (huggingface.co)... 18.238.109.52, 18.238.109.121, 18.238.109.102, ...
Connecting to huggingface.co (huggingface.co)|18.238.109.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983900 (961K) [text/plain]
Saving to: ‘MedText_zhtw.json’

MedText_zhtw.json   100%[===================>] 960.84K  5.37MB/s    in 0.2s    

2024-08-13 01:43:18 (5.37 MB/s) - ‘MedText_zhtw.json’ saved [983900/983900]



In [ ]:
%cd /content/LLaMA-Factory/data/

/content/LLaMA-Factory/data


In [ ]:
import json

# 讀取 dataset_info.json 文件
with open('dataset_info.json', 'r') as file:
    data = json.load(file)

# 新增 "Medtext" 項目
data["Medtext"] = {
    "file_name": "MedText_zhtw.json"
}

# 將修改後的內容保存回 dataset_info.json 文件
with open('dataset_info.json', 'w') as file:
    json.dump(data, file, indent=2, ensure_ascii=False)


In [ ]:
%cd /content/LLaMA-Factory/
!GRADIO_SHARE=1 llamafactory-cli webui

Streaming output truncated to the last 5000 lines.
model-00013-of-00030.safetensors:  62% 3.08G/5.00G [00:25<00:15, 121MB/s]
model-00013-of-00030.safetensors:  62% 3.10G/5.00G [00:25<00:14, 127MB/s]
model-00013-of-00030.safetensors:  62% 3.12G/5.00G [00:25<00:14, 130MB/s]
model-00013-of-00030.safetensors:  63% 3.15G/5.00G [00:26<00:13, 133MB/s]
model-00013-of-00030.safetensors:  63% 3.17G/5.00G [00:26<00:13, 135MB/s]
model-00013-of-00030.safetensors:  64% 3.19G/5.00G [00:26<00:13, 136MB/s]
model-00013-of-00030.safetensors:  64% 3.21G/5.00G [00:26<00:13, 136MB/s]
model-00013-of-00030.safetensors:  65% 3.23G/5.00G [00:26<00:12, 137MB/s]
model-00013-of-00030.safetensors:  65% 3.25G/5.00G [00:26<00:12, 137MB/s]
model-00013-of-00030.safetensors:  65% 3.27G/5.00G [00:27<00:12, 137MB/s]
model-00013-of-00030.safetensors:  66% 3.29G/5.00G [00:27<00:12, 138MB/s]
model-00013-of-00030.safetensors:  66% 3.31G/5.00G [00:27<00:12, 138MB/s]
model-00013-of-00030.safetensors:  67% 3.33G/5.00G [00:27<00:

## 使用命令行微调模型

微调过程大约需要 30 分钟。

In [ ]:
import json

args = dict(
  stage="sft",                        # 进行指令监督微调
  do_train=True,
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # 使用 4 比特量化版 Llama-3-8b-Instruct 模型
  dataset="identity,alpaca_en_demo,alpaca_zh_demo",      # 使用 alpaca 和自我认知数据集
  template="llama3",                     # 使用 llama3 提示词模板
  finetuning_type="lora",                   # 使用 LoRA 适配器来节省显存
  lora_target="all",                     # 添加 LoRA 适配器至全部线性层
  output_dir="llama3_lora",                  # 保存 LoRA 适配器的路径
  per_device_train_batch_size=2,               # 批处理大小
  gradient_accumulation_steps=4,               # 梯度累积步数
  lr_scheduler_type="cosine",                 # 使用余弦学习率退火算法
  logging_steps=10,                      # 每 10 步输出一个记录
  warmup_ratio=0.1,                      # 使用预热学习率
  save_steps=1000,                      # 每 1000 步保存一个检查点
  learning_rate=5e-5,                     # 学习率大小
  num_train_epochs=3.0,                    # 训练轮数
  max_samples=300,                      # 使用每个数据集中的 300 条样本
  max_grad_norm=1.0,                     # 将梯度范数裁剪至 1.0
  quantization_bit=4,                     # 使用 4 比特 QLoRA
  loraplus_lr_ratio=16.0,                   # 使用 LoRA+ 算法并设置 lambda=16.0
  fp16=True,                         # 使用 float16 混合精度训练
)

json.dump(args, open("train_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli train train_llama3.json

## 模型推理

In [ ]:
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc

%cd /content/LLaMA-Factory/

args = dict(
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # 使用 4 比特量化版 Llama-3-8b-Instruct 模型
  adapter_name_or_path="llama3_lora",            # 加载之前保存的 LoRA 适配器
  template="llama3",                     # 和训练保持一致
  finetuning_type="lora",                  # 和训练保持一致
  quantization_bit=4,                    # 加载 4 比特量化模型
)
chat_model = ChatModel(args)

messages = []
print("使用 `clear` 清除对话历史，使用 `exit` 退出程序。")
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break
  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("对话历史已清除")
    continue

  messages.append({"role": "user", "content": query})
  print("Assistant: ", end="", flush=True)

  response = ""
  for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response})

torch_gc()

## 合并 LoRA 权重并上传模型

注意：Colab 免费版仅提供了 12GB 系统内存，而合并 8B 模型的 LoRA 权重需要至少 18GB 系统内存，因此你 **无法** 在免费版运行此功能。

In [ ]:
!huggingface-cli login

In [ ]:
import json

args = dict(
  model_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct", # 使用非量化的官方 Llama-3-8B-Instruct 模型
  adapter_name_or_path="llama3_lora",            # 加载之前保存的 LoRA 适配器
  template="llama3",                     # 和训练保持一致
  finetuning_type="lora",                  # 和训练保持一致
  export_dir="llama3_lora_merged",              # 合并后模型的保存目录
  export_size=2,                       # 合并后模型每个权重文件的大小（单位：GB）
  export_device="cpu",                    # 合并模型使用的设备：`cpu` 或 `cuda`
  #export_hub_model_id="your_id/your_model",         # 用于上传模型的 HuggingFace 模型 ID
)

json.dump(args, open("merge_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli export merge_llama3.json